In [262]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow import keras
from tensorflow.keras import layers

In [263]:
data = pd.read_csv("BELGAVI(Working_data2).csv")

In [264]:
X_basic = data[['Temperature', 'Humidity', 'Precipitation', 'NDVI']]
y_sparse = data['Sparse']
y_dense = data['Dense']

X_all = data[['Temperature', 'Humidity', 'Precipitation', 'NDVI', 'Sparse', 'Dense']]

In [265]:
scaler_basic = MinMaxScaler()
X_scaled_basic = scaler_basic.fit_transform(X_basic)

scaler_all = MinMaxScaler()
X_scaled_all = scaler_all.fit_transform(X_all)

In [266]:
# Deep learning model
def build_model(input_dim):
    model = keras.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [267]:
model_basic_sparse = build_model(X_scaled_basic.shape[1])
model_basic_dense = build_model(X_scaled_basic.shape[1])
model_basic_sparse.fit(X_scaled_basic, y_sparse, epochs=50, verbose=0)
model_basic_dense.fit(X_scaled_basic, y_dense, epochs=50, verbose=0)
y_basic_sparse_pred = model_basic_sparse.predict(X_scaled_basic)
y_basic_dense_pred = model_basic_dense.predict(X_scaled_basic)


model_all_sparse = build_model(X_scaled_all.shape[1])
model_all_dense = build_model(X_scaled_all.shape[1])
model_all_sparse.fit(X_scaled_all, y_sparse, epochs=50, verbose=0)
model_all_dense.fit(X_scaled_all, y_dense, epochs=50, verbose=0)
y_all_sparse_pred = model_all_sparse.predict(X_scaled_all)
y_all_dense_pred = model_all_dense.predict(X_scaled_all)


1/1 [==============================] - 0s 46ms/step


In [268]:
diff_basic_sparse = y_sparse.values - y_basic_sparse_pred.flatten()
diff_basic_dense = y_dense.values - y_basic_dense_pred.flatten()

diff_all_sparse = y_sparse.values - y_all_sparse_pred.flatten()
diff_all_dense = y_dense.values - y_all_dense_pred.flatten()

In [269]:
mse_basic_sparse = mean_squared_error(y_sparse, y_basic_sparse_pred.flatten())
mse_basic_dense = mean_squared_error(y_dense, y_basic_dense_pred.flatten())
mse_all_sparse = mean_squared_error(y_sparse, y_all_sparse_pred.flatten())
mse_all_dense = mean_squared_error(y_dense, y_all_dense_pred.flatten())


In [270]:
results_basic = pd.DataFrame({
    'Year': data['Year'],
    'Month': data['Month'],
    'Temperature': X_basic['Temperature'],
    'Humidity': X_basic['Humidity'],
    'Precipitation': X_basic['Precipitation'],
    'NDVI': X_basic['NDVI'],
    'Actual_Sparse': y_sparse.values,
    'Actual_Dense': y_dense.values,
    'Predicted_Sparse': y_basic_sparse_pred.flatten(),
    'Predicted_Dense': y_basic_dense_pred.flatten(),
    'Difference_Sparse': diff_basic_sparse,
    'Difference_Dense': diff_basic_dense
})

results_all = pd.DataFrame({
    'Year': data['Year'],
    'Month': data['Month'],
    'Temperature': X_all['Temperature'],
    'Humidity': X_all['Humidity'],
    'Precipitation': X_all['Precipitation'],
    'NDVI': X_all['NDVI'],
    'Sparse': X_all['Sparse'],
    'Dense': X_all['Dense'],
    'Predicted_Sparse': y_all_sparse_pred.flatten(),
    'Predicted_Dense': y_all_dense_pred.flatten(),
    'Difference_Predicted_Sparse': diff_all_sparse,
    'Difference_Predicted_Dense': diff_all_dense
})

results_basic.to_csv("output_results_basic.csv", index=False)
results_all.to_csv("output_results_all.csv", index=False)

In [271]:
print("MSE Basic Sparse:", mse_basic_sparse)
print("MSE Basic Dense:", mse_basic_dense)

print("MSE ALL Sparse:", mse_all_sparse)
print("MSE ALL Dense:", mse_all_dense)



MSE Basic Sparse: 0.005036324983272563
MSE Basic Dense: 0.015643575681511382
MSE ALL Sparse: 0.0004115328002712328
MSE ALL Dense: 0.0005832203469104159


In [272]:
results_basic

,Year,Month,Temperature,Humidity,Precipitation,NDVI,Actual_Sparse,Actual_Dense,Predicted_Sparse,Predicted_Dense,Difference_Sparse,Difference_Dense
0,2022,January,20.350000,10.203548,0.007419,0.3592,0.312935,0.565748,0.209391,0.441674,0.103544,0.124074
1,2022,February,23.583214,8.817143,0.100000,0.3343,0.311759,0.568453,0.220070,0.474618,0.091689,0.093835
2,2022,March,27.772258,10.454839,0.399677,0.3683,0.312681,0.576273,0.319077,0.626562,-0.006396,-0.050289
3,2022,April,28.393667,13.556333,1.731333,0.4760,0.315724,0.619876,0.355850,0.734408,-0.040126,-0.114532
4,2022,May,27.188387,15.795161,2.238065,0.4384,0.315115,0.621837,0.367560,0.669445,-0.052445,-0.047608
5,2022,June,24.565333,16.935000,10.234667,0.3255,0.314887,0.611736,0.240941,0.485904,0.073946,0.125832
6,2022,July,23.260000,17.528710,16.915484,0.2000,0.100000,0.100000,0.188734,0.302079,-0.088734,-0.202079
7,2022,August,22.915484,17.172903,11.845161,0.2181,0.314367,0.636269,0.199862,0.324852,0.114505,0.311417
8,2022,September,22.895333,17.031333,8.506333,0.5143,0.315279,0.672559,0.308373,0.686262,0.006906,-0.013703
9,2022,October,22.287419,15.445161,4.814516,0.5691,0.316090,0.684067,0.333142,0.782425,-0.017052,-0.098358


In [273]:
results_all

,Year,Month,Temperature,Humidity,Precipitation,NDVI,Sparse,Dense,Predicted_Sparse,Predicted_Dense,Difference_Predicted_Sparse,Difference_Predicted_Dense
0,2022,January,20.350000,10.203548,0.007419,0.3592,0.312935,0.565748,0.332317,0.567201,-0.019382,-0.001453
1,2022,February,23.583214,8.817143,0.100000,0.3343,0.311759,0.568453,0.317048,0.577008,-0.005289,-0.008555
2,2022,March,27.772258,10.454839,0.399677,0.3683,0.312681,0.576273,0.327620,0.582933,-0.014939,-0.006660
3,2022,April,28.393667,13.556333,1.731333,0.4760,0.315724,0.619876,0.305112,0.662752,0.010612,-0.042876
4,2022,May,27.188387,15.795161,2.238065,0.4384,0.315115,0.621837,0.305183,0.624316,0.009932,-0.002479
5,2022,June,24.565333,16.935000,10.234667,0.3255,0.314887,0.611736,0.325780,0.645169,-0.010893,-0.033433
6,2022,July,23.260000,17.528710,16.915484,0.2000,0.100000,0.100000,0.085600,0.147519,0.014400,-0.047519
7,2022,August,22.915484,17.172903,11.845161,0.2181,0.314367,0.636269,0.339971,0.636570,-0.025604,-0.000301
8,2022,September,22.895333,17.031333,8.506333,0.5143,0.315279,0.672559,0.339021,0.721700,-0.023742,-0.049141
9,2022,October,22.287419,15.445161,4.814516,0.5691,0.316090,0.684067,0.337367,0.695137,-0.021277,-0.011070
